## UAV fleet Simulation 

This is for testing propouses

In [1]:
import os
import subprocess
import numpy as np

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns
from helpers.visualization.gazebo_world import update_world
from helpers.change_coordinates import local2global,heading_to_yaw,find_spawns
from helpers.visualization.plots import plot_3d_interactive

from plan import Plan,State

Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/runway.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

## Choose Simulator

In [4]:
simulator ='gazebo' #'QGroundControl' # 

## Create Plans

In [5]:
offsets = [ #east, north, up, heading
    (5, 5, 0, 90), 
    (10, 0, 0, 45),
    (-5, -10, 0, 225),
    (-15, 0, 0, 0),
    (0, -20, 0, 0),
]

n_uavs = len(offsets)


local_paths=[Plan.create_square_path(side_len=dist,alt= 5) for dist in (5,7,10,12,15)]

homes=np.array([offset[:3] for offset in offsets])

global_paths=[local2global(np.array(path), home,pairwise=True) for path,home in zip(local_paths,homes)]

plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5) for path in local_paths]


colors=['Green','Blue','Red','Orange','Yellow',]
markers = {f'uav_{i+1}': {'pos':global_paths[i],'color':colors[i]} for i in range(n_uavs)}


if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,0)
    spawns=find_spawns(home_position, offsets)

plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)



Plan 'basic plan' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to -> (0, 5, 5)' created — no connection yet 🧩
Step 'go to -> (5, 5, 5)' created — no connection yet 🧩
Step 'go to -> (5, 0, 5)' created — no connection yet 🧩
Step 'go to -> (0, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'ba

Launch Simulator

In [6]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

Launch Vehicles

In [7]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {ardupilot_vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

In [8]:
from pymavlink import mavutil
conn=mavutil.mavlink_connection(f'udp:127.0.0.1:14550')
conn.wait_heartbeat()

Create UAVs

In [9]:
uavs=[]
for i,(plan,home) in enumerate(zip(plans,homes)):
    uavs.append(VehicleLogic(sys_id=i+1,
                    home=home,
                    plan= plan))

vehicle 1 created
vehicle 2 created
vehicle 3 created
vehicle 4 created
vehicle 5 created


## Execute Plan

In [10]:
mission_incomplete = set(range(n_uavs))
while mission_incomplete:
    to_remove = set()
    for i in mission_incomplete:
        uav = uavs[i]
        if uav.plan.state == State.DONE:
            to_remove.add(i)
        else:
            uav.act_plan()
    mission_incomplete -= to_remove  # batch-remove outside the loop
    

Vehicle 1: Plan 'basic plan' is now connected ✅🔗
Vehicle 1: ▶️ Starting Plan: basic plan
Vehicle 2: Plan 'basic plan' is now connected ✅🔗
Vehicle 2: ▶️ Starting Plan: basic plan
Vehicle 3: Plan 'basic plan' is now connected ✅🔗
Vehicle 3: ▶️ Starting Plan: basic plan
Vehicle 4: Plan 'basic plan' is now connected ✅🔗
Vehicle 4: ▶️ Starting Plan: basic plan
Vehicle 5: Plan 'basic plan' is now connected ✅🔗
Vehicle 5: ▶️ Starting Plan: basic plan
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: ▶️ Starting Action: Pre-Arm Check
Vehicle 2: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 2: ▶️ Starting Action: Pre-Arm Check
Vehicle 3: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 3: ▶️ Starting Action: Pre-Arm Check
Vehicle 4: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 4: ▶️ Starting Action: Pre-Arm Check
Vehicle 5: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 5: ▶️ Starting Action: Pre-Arm Check
Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle

KeyboardInterrupt: 

## Kill all related process

In [ ]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")